<h1><center>Minimal Covers</center></h1>

Devise and implement three Python functions that compute a minimal cover, all accessible (by the algorithm given in the lecture) minimal covers, and all possible minimal covers of a given list of functional dependencies, respectively. 
Complete the provided Jupyter Notebook using the Python 3 kernel. 
Do not change the name and the parameters of the functions in the template. 
You may add ancillary functions if necessary or convenient.
You may import additional Python standard libraries but no external libraries.

For all questions, use the following data structures.
A functional dependency is represented as a list of two lists. For
example, the functional dependency {A, B} → {C} is represented as [['A', 'B'], ['C']]. 
A set of functional dependencies is represented as a list of functional dependencies. 

Ensure that your functions process the input and produce the output in the format specified in the corresponding questions.

For simplicity, you may assume that the input is always valid (e.g. no empty string, no incomplete functional dependency).
Consider the case in which a functional dependency's left- or right-hand side is empty.

The code should be readable and adequately commented.

The marking team considers clarity of the code and comments, quality of the algorithm and  implementation, and correctness and efficiency of the code.

### Indicate your student number: A0290880X

# Question 1 (4 points)
Write a function *min_cover* that takes a list of functional dependencies as input and returns a minimal cover of the functional dependencies, for example,
````
min_cover([[['A'], ['B', 'C']],[['B'], ['C','D']], [['D'], ['B']], [['A','B','E'], ['F']]])
outputs: [[['A'], ['B']], [['B'], ['C']], [['B'], ['D']], [['D'], ['B']], [['A', 'E'], ['F']]]
````
The results is a minimal cover, i.e. a list of functional dependencies.
There might be more than one correct answer to this question.

In [1]:
def get_all_attributes(fds: list[list]):
    """
    Generate all attributes from 'A'
    """
    max_char = ''
    for fd in fds:
        lhs, rhs = fd
        for char in ''.join(lhs + rhs):
            if char > max_char:
                max_char = char
    # found max_char. e.g. F
    return [chr(char_code) for char_code in range(ord('A'), ord(max_char) + 1)]

fds = [[['A'], ['B', 'C']],[['B'], ['C','D']], [['D'], ['B']], [['A','B','E'], ['F']]]
get_all_attributes(fds)

['A', 'B', 'C', 'D', 'E', 'F']

In [2]:
def cal_attribute_closure(attributes: set, fds: list[list]):
    """
    It is a fix point algorithm
    attribute: set of attribute like {'A', 'E'}
    fds: a list of fd
    """
    closure = attributes.copy()
    changed = True
    while changed:
        changed = False
        for lhs, rhs in fds:
            if set(lhs).issubset(closure) and not set(rhs).issubset(closure):
                # union
                closure.update(rhs)
                changed = True
    return closure

fds = [[['A'], ['B', 'C']], [['B'], ['C','D']], [['D'], ['B']], [['A','B','E'], ['F']]]
attributes = {'A'}
cal_attribute_closure(attributes, fds)

{'A', 'B', 'C', 'D'}

In [3]:
from itertools import combinations
from collections import OrderedDict

def cal_combination_attribute_closure(all_attributes: list, fds: list[list]):
    """
    Generate combinations of attributes untill superkey is found
    """
    combs = OrderedDict()
    superkeys = []
    for i in range(len(all_attributes) - 1):
        found_superkey = False
        for combination in combinations(all_attributes, i + 1):
            comb = combination
            attribute_closure = cal_attribute_closure(set(comb), fds)
            combs[tuple(sorted(comb))] = attribute_closure
            # check if comb is superkey
            if attribute_closure == set(all_attributes):
                superkeys.append(comb)
                found_superkey = True
                break
        if found_superkey: break
    return superkeys, combs

fds = [[['A'], ['B', 'C']], [['B'], ['C','D']], [['D'], ['B']], [['A','B','E'], ['F']]]
attributes = get_all_attributes(fds)
cal_combination_attribute_closure(attributes, fds)

([('A', 'E')],
 OrderedDict([(('A',), {'A', 'B', 'C', 'D'}),
              (('B',), {'B', 'C', 'D'}),
              (('C',), {'C'}),
              (('D',), {'B', 'C', 'D'}),
              (('E',), {'E'}),
              (('F',), {'F'}),
              (('A', 'B'), {'A', 'B', 'C', 'D'}),
              (('A', 'C'), {'A', 'B', 'C', 'D'}),
              (('A', 'D'), {'A', 'B', 'C', 'D'}),
              (('A', 'E'), {'A', 'B', 'C', 'D', 'E', 'F'})]))

In [4]:
def is_redundant(fd, fds):
    """
    Check if fd can be derived from other fds
    :param fd: the fd we need to check
    :param fds: other fds except fd
    :return: bool
    """
    remaining_fds = fds.copy()
    # remove fd from fds first
    remaining_fds.remove(fd)
    X, Y = fd
    closure = cal_attribute_closure(set(X), remaining_fds)
    # after apply other fds, wether we find X->Y, which is Y in closure of X 
    return set(Y).issubset(closure)

fd = [['A'], ['C']]
fds = [[['B'], ['C']], [['A'], ['C']], [['A'], ['B']], [['B'], ['D']], [['D'], ['B']], [['A', 'E'], ['F']]]
is_redundant(fd, fds)

True

In [5]:
def remove_duplicate_fds(fds):
    """
    Remove duplicate fds in fds and maintain the order
    :param fds: a list of functional dependencies
    :return: new list of functional dependencies without duplicate
    """
    seen = set()
    new_fds = []
    for fd in fds:
        str_item = str(fd)  # Convert inner lists to strings for set comparison
        if str_item not in seen:
            seen.add(str_item)
            new_fds.append(fd)
    return new_fds

fds = [[['A'], ['B']], [['B'], ['A']], [['B'], ['C']], [['A'], ['C']], [['A'], ['C']]]
remove_duplicate_fds(fds)

[[['A'], ['B']], [['B'], ['A']], [['B'], ['C']], [['A'], ['C']]]

In [6]:
def min_cover(fds):
    print(f"Original dependencies: \n{fds}")
    all_attributes = get_all_attributes(fds)

    # Step 1: Decompose the right-hand side (RHS) of each dependency
    fds1 = []
    for fd in fds:
        lhs, rhs = fd
        for rhs_attribute in rhs:
            fds1.append([lhs, [rhs_attribute]])
    print(f"After minimize RHS: \n{fds1}")

    # Step 2: Minimize the left-hand side (LHS) of each dependency
    # Calculate attribute closure
    _, attribute_closures = cal_combination_attribute_closure(all_attributes, fds1)
    fds2 = []
    for fd in fds1:
        lhs, rhs = fd
        # remove trivial dependency, e.g. A,B->A
        if set(rhs).issubset(set(lhs)):
            continue
        # lhs is already minimized
        if len(lhs) == 1:
            fds2.append(fd)
        # lhs is not minimized
        else:
            # find first attribute closure which contains rhs
            for attributes_set, closure in attribute_closures.items():
                # e.g. 'F' should not be in attributes_set, this is trivial
                if set(rhs).issubset(closure) and (not set(rhs).issubset(attributes_set)):
                    fds2.append([sorted(list(attributes_set)), rhs])
                    break
    # remove duplicate fd
    fds2 = remove_duplicate_fds(fds2)
    print(f"After minimize LHS: \n{fds2}")

    # Step 3: Check if each fd is redundant. i.e. fd can be derived from other fds
    # Only need to find one solution
    min_cover = []
    remaining_fds2 = fds2.copy()
    for fd in fds2:
        if is_redundant(fd, remaining_fds2):
            remaining_fds2.remove(fd)
        else:
            min_cover.append(fd)
    print(f"After remove redundant: \n{min_cover}")
    return min_cover

### Expected Result

In [7]:
fds = [[['A'], ['B','C']],[['B'], ['C','D']], [['D'], ['B']], [['A','B','E'], ['F']]]

# test case in ppt
# fds = [[['B'], ['D']], [['B'], ['C']], [['C'], ['B']], [['C'], ['D']], [['B'], ['E']], [['C'], ['E']]]

min_cover(fds)

Original dependencies: 
[[['A'], ['B', 'C']], [['B'], ['C', 'D']], [['D'], ['B']], [['A', 'B', 'E'], ['F']]]
After minimize RHS: 
[[['A'], ['B']], [['A'], ['C']], [['B'], ['C']], [['B'], ['D']], [['D'], ['B']], [['A', 'B', 'E'], ['F']]]
After minimize LHS: 
[[['A'], ['B']], [['A'], ['C']], [['B'], ['C']], [['B'], ['D']], [['D'], ['B']], [['A', 'E'], ['F']]]
After remove redundant: 
[[['A'], ['B']], [['B'], ['C']], [['B'], ['D']], [['D'], ['B']], [['A', 'E'], ['F']]]


[[['A'], ['B']],
 [['B'], ['C']],
 [['B'], ['D']],
 [['D'], ['B']],
 [['A', 'E'], ['F']]]

**The result you get should be similar to:**
```
[[['A'], ['B']],
 [['B'], ['C']],
 [['B'], ['D']],
 [['D'], ['B']],
 [['A', 'E'], ['F']]]
```

# Question 2 (3 points) 
Write a function *min_covers* that takes a list of functional dependencies as input and returns all minimal covers reachable from the given list of functional dependencies, for example,
````
min_cover([[['A'], ['A', 'B']], [['B'], ['A','C']], [['A'], ['C']], [['A','B'], ['C']]])
outputs: [
            [[['A'], ['B']], [['B'], ['A']], [['B'], ['C']]],
            [[['A'], ['B']], [['B'], ['A']], [['A'], ['C']]]
        ]
````
The results is a list of minimal covers.
Make sure that you clearly explain the rationale of your solution in comments of the code.

In [8]:
def generate_all_min_covers(fds: list[list]) -> list[list]:
    """
    To minimize fds itself (remove redundant in all possible ways)
    :param fds: 
    :return: 
    """
    all_min_covers = []

    def backtrack(cur_min_cover, remaining_fds, index):
        if index == len(remaining_fds):
            # remaining fds cannot contain redundant
            no_redundant_fd = True
            for fd in remaining_fds:
                if is_redundant(fd, remaining_fds):
                    no_redundant_fd = False
                    break
            if no_redundant_fd: all_min_covers.append(cur_min_cover)
            return
        
        cur_fd = remaining_fds[index]
        if is_redundant(cur_fd, remaining_fds):
            # don't select cur_min_cover
            backtrack(cur_min_cover, remaining_fds[:index] + remaining_fds[index+1:], index)
        # select cur_min_cover
        backtrack(cur_min_cover+[cur_fd], remaining_fds, index+1)
        
    backtrack([], fds, 0)
    return all_min_covers

In [9]:
def min_covers(fds):
    print(f"Original dependencies: \n{fds}")
    all_attributes = get_all_attributes(fds)

    # Step 1: Decompose the right-hand side (RHS) of each dependency
    fds1 = []
    for fd in fds:
        lhs, rhs = fd
        for rhs_attribute in rhs:
            fds1.append([lhs, [rhs_attribute]])
    print(f"After minimize RHS: \n{fds1}")

    # Step 2: Minimize the left-hand side (LHS) of each dependency
    # Calculate attribute closure
    _, attribute_closures = cal_combination_attribute_closure(all_attributes, fds1)
    fds2 = []
    for fd in fds1:
        lhs, rhs = fd
        # remove trivial dependency, e.g. A,B->A
        if set(rhs).issubset(set(lhs)):
            continue
        # lhs is already minimized
        if len(lhs) == 1:
            fds2.append(fd)
        # lhs is not minimized
        else:
            # find first attribute closure which contains rhs
            for attributes_set, closure in attribute_closures.items():
                # e.g. 'F' should not be in attributes_set, this is trivial
                if set(rhs).issubset(closure) and (not set(rhs).issubset(attributes_set)):
                    fds2.append([sorted(list(attributes_set)), rhs])
                    break
    # remove duplicate fd
    fds2 = remove_duplicate_fds(fds2)
    print(f"After minimize LHS: \n{fds2}")

    # Step 3: Check if fd is redundant. i.e. can be derived from other fds
    min_covers = generate_all_min_covers(fds2)        
    print(f"After remove redundant, multiple possible min_covers: \n{min_covers}")
    return min_covers

### Expected Result

In [10]:
fds = [[['A'], ['A', 'B']], [['B'], ['A','C']], [['A'], ['C']], [['A','B'], ['C']]]

# test case in ppt
# fds = [[['B'], ['D']], [['B'], ['C']], [['C'], ['B']], [['C'], ['D']], [['B'], ['E']], [['C'], ['E']]]

min_covers(fds)

Original dependencies: 
[[['A'], ['A', 'B']], [['B'], ['A', 'C']], [['A'], ['C']], [['A', 'B'], ['C']]]
After minimize RHS: 
[[['A'], ['A']], [['A'], ['B']], [['B'], ['A']], [['B'], ['C']], [['A'], ['C']], [['A', 'B'], ['C']]]
After minimize LHS: 
[[['A'], ['B']], [['B'], ['A']], [['B'], ['C']], [['A'], ['C']]]
After remove redundant, multiple possible min_covers: 
[[[['A'], ['B']], [['B'], ['A']], [['A'], ['C']]], [[['A'], ['B']], [['B'], ['A']], [['B'], ['C']]]]


[[[['A'], ['B']], [['B'], ['A']], [['A'], ['C']]],
 [[['A'], ['B']], [['B'], ['A']], [['B'], ['C']]]]

**The result you get should be similar to:**
```
[
    [[['A'], ['B']], [['B'], ['A']], [['B'], ['C']]],
    [[['A'], ['B']], [['B'], ['A']], [['A'], ['C']]]
]
```

# Question 3 (3 points)
* Write a function *all_min_covers* that takes a list of functional dependencies as input and returns all minimal covers, for example,
````
all_min_covers([[['A'], ['B']],[['B'], ['C']], [['C'], ['A']]])
outputs: [
	[[['A'], ['C']], [['B'], ['A']], [['C'], ['A']], [['A'], ['B']]],
	[[['B'], ['C']], [['C'], ['A']], [['A'], ['B']]],
	[[['C'], ['B']], [['A'], ['C']], [['C'], ['A']], [['B'], ['C']]],
	[[['C'], ['B']], [['A'], ['C']], [['B'], ['A']]],
	[[['B'], ['C']], [['A'], ['B']], [['B'], ['A']], [['C'], ['B']]]
]
````

The results is a list of minimal covers.
Make sure that you clearly explain the rationale of your solution in comments of the code.
This question is challenging and not everyone may find a solution.

In [11]:
def generate_all_reachable_pairs(fds):
    """
    Use dfs to get each node's all reachable node
    :param fds: 
    :return: 
    """
    graph = {}
    reachable_pairs = []

    # Build the graph
    for fd in fds:
        X, Y = fd
        if X[0] not in graph:
            graph[X[0]] = set()
        graph[X[0]].add(Y[0])

    # Perform DFS
    def dfs(visited, start_node, current_node):
        visited.add(start_node)
        for neighbor in graph.get(current_node, []):
            if neighbor not in visited:
                reachable_pairs.append([[start_node], [neighbor]])
                dfs(visited, start_node, neighbor)

    for node in graph.keys():
        # generate pairs where A can reach
        visited = set()
        dfs(visited, node, node)

    return reachable_pairs

fds = [[['A'], ['B']],[['B'], ['C']], [['C'], ['A']]]
generate_all_reachable_pairs(fds)

[[['A'], ['B']],
 [['A'], ['C']],
 [['B'], ['C']],
 [['B'], ['A']],
 [['C'], ['A']],
 [['C'], ['B']]]

In [12]:
def all_min_covers(fds):
    # graph dfs, start from A, can reach B,C; start from B, can reach A,C; start from C, can reach A,B
    # we have [[['A'],['B']], [['A'],['C']], [['B'],['C']], [['B'],['A']], [['C'],['A']], [['C'],['B']]]
    # generate its min covers
    fds1 = generate_all_reachable_pairs(fds)
    all_min_covers = min_covers(fds1)
    return all_min_covers

### Expected Result

In [13]:
fds = [[['A'], ['B']],[['B'], ['C']], [['C'], ['A']]]
all_min_covers(fds)

Original dependencies: 
[[['A'], ['B']], [['A'], ['C']], [['B'], ['C']], [['B'], ['A']], [['C'], ['A']], [['C'], ['B']]]
After minimize RHS: 
[[['A'], ['B']], [['A'], ['C']], [['B'], ['C']], [['B'], ['A']], [['C'], ['A']], [['C'], ['B']]]
After minimize LHS: 
[[['A'], ['B']], [['A'], ['C']], [['B'], ['C']], [['B'], ['A']], [['C'], ['A']], [['C'], ['B']]]
After remove redundant, multiple possible min_covers: 
[[[['A'], ['C']], [['B'], ['A']], [['C'], ['B']]], [[['A'], ['C']], [['B'], ['C']], [['C'], ['A']], [['C'], ['B']]], [[['A'], ['B']], [['B'], ['C']], [['C'], ['A']]], [[['A'], ['B']], [['B'], ['C']], [['B'], ['A']], [['C'], ['B']]], [[['A'], ['B']], [['A'], ['C']], [['B'], ['A']], [['C'], ['A']]]]


[[[['A'], ['C']], [['B'], ['A']], [['C'], ['B']]],
 [[['A'], ['C']], [['B'], ['C']], [['C'], ['A']], [['C'], ['B']]],
 [[['A'], ['B']], [['B'], ['C']], [['C'], ['A']]],
 [[['A'], ['B']], [['B'], ['C']], [['B'], ['A']], [['C'], ['B']]],
 [[['A'], ['B']], [['A'], ['C']], [['B'], ['A']], [['C'], ['A']]]]

**The result you get should be:**
```
[
	[[['A'], ['C']], [['B'], ['A']], [['C'], ['A']], [['A'], ['B']]],
	[[['B'], ['C']], [['C'], ['A']], [['A'], ['B']]],
	[[['C'], ['B']], [['A'], ['C']], [['C'], ['A']], [['B'], ['C']]],
	[[['C'], ['B']], [['A'], ['C']], [['B'], ['A']]],
	[[['B'], ['C']], [['A'], ['B']], [['B'], ['A']], [['C'], ['B']]]
]
````